# Tarefa 02

---

**1.** Monte um passo a passo para o algoritmo RF

Similar ao Bagging, o Random Forest segue os seguintes passos:

1. Bootstrap + Seleção de Características: Da mesma forma que o Bagging, o Random Forest utiliza amostras aleatórias com reposição do conjunto de dados original. Contudo, em cada uma dessas amostras, apenas um subconjunto aleatório de variáveis é selecionado (feature selection). Para problemas de classificação, é comum escolher a raiz quadrada do número total de variáveis, enquanto em problemas de regressão, é usual selecionar um terço das variáveis.

2. Modelagem com Árvores de Decisão: Nesta etapa, um modelo de Machine Learning, especificamente uma árvore de decisão, é treinado de forma independente em cada amostra bootstrap, utilizando as variáveis aleatórias definidas no passo anterior.

3. Agregação: Por fim, os resultados de cada modelo independente (cada árvore de decisão) são agregados para obter uma previsão final. Em problemas de classificação, a agregação geralmente é realizada por meio do voto majoritário, onde a classe mais prevista pelos modelos individuais é escolhida como a classe final. Em problemas de regressão, a agregação é feita calculando a média das previsões dos modelos individuais.

**2.** Explique com suas palavras o Random Forest

O Random Forest pode ser conceituado como uma ampliação do Bootstrap Aggregating, também conhecido como Bagging, oferecendo melhorias em desempenho e resultados. Assim como no Bagging, o Random Forest é um método de combinação de modelos de Machine Learning, onde cada modelo é treinado em variações do conjunto de dados original. Essas variações, denominadas amostras bootstrap, consistem em subconjuntos aleatórios do conjunto de dados original, com possíveis repetições, mantendo o mesmo número de observações. No entanto, o Random Forest se diferencia ao utilizar apenas uma quantidade determinada e aleatória de variáveis em cada modelo, especialmente em modelos de árvore de decisão. Esta abordagem tem como objetivo reduzir a variância dos resultados e mitigar o risco de overfitting. Para obter a previsão final, a agregação dos modelos é realizada de forma semelhante ao Bagging, onde a média é empregada para regressão e votação para classificação.

**3.** Qual a diferença entre Bagging e Random Forest?


A distinção entre Bagging e Random Forest reside no desempenho e na redução da variância entre os resultados dos modelos. Bagging é uma técnica mais ampla de combinação de modelos de Machine Learning que utiliza amostragem aleatória, enquanto Random Forest é uma variação mais específica desse método que emprega árvores de decisão. No Random Forest, cada modelo é treinado em subconjuntos aleatórios do conjunto de dados original, com uma quantidade reduzida e aleatória de variáveis consideradas em cada modelo. Essa abordagem aumenta a robustez do modelo e, ao combinar os resultados dos modelos, a previsão final tende a ser mais precisa do que no Bagging. Portanto, Random Forest é uma extensão aprimorada do Bagging, oferecendo um desempenho superior e uma redução adicional na variância dos resultados.

**Em resumo, ``Random Forest`` funciona melhor que o Bagging, pois as árvores amostradas são mais independentes (menor correlação).**

**4.** (Opcional) Implementar em python o Random Forest
> - Bootstrap
> - Feature selection
> - Modelagem com Decision trees
> - Agregação

In [1]:
# Import das bibliotecas:

import numpy  as np
import pandas as pd

from sklearn.datasets        import load_iris
from sklearn.datasets        import load_diabetes

from sklearn.model_selection import train_test_split

from sklearn.tree            import DecisionTreeClassifier
from sklearn.metrics         import accuracy_score

from sklearn.tree            import DecisionTreeRegressor
from sklearn.metrics         import mean_squared_error, r2_score

In [2]:
# Exemplo da técnica Random Forest para problemas de classificação:

X = load_iris().data
y = load_iris().target

df = pd.DataFrame(X, columns=load_iris().feature_names)
df['target'] = y

def rf_classifier(df:pd.DataFrame, 
                  num_bootstrap_samples:int=3,  # Parâmetro da função que define a quantidade de amostragens para treinamento
                  test_size:float=0.25
                 ) -> pd.DataFrame:
    
    df_train, df_test = train_test_split(df, test_size=test_size)
    
    X_test = df_test.drop(['target'], axis=1)
    y_test = df_test['target'].rename('y_test')
    
    # Dicionário para os resultados das predições de cada modelo
    y_pred_bagging = {}

    for i in range(num_bootstrap_samples):
        # Bootstrap
        df_train = df_train.sample(n=len(df_train), 
                                   replace=True)  # Amostragem COM reposição

        X_train = df_train.drop(['target'], axis=1)
        # Feature selection
        X_train = X_train.sample(n=round(np.sqrt(X_train.shape[1])),  # Cálculo da raiz quadrada da quantidade de variáveis
                                 axis=1)
        
        y_train = df_train['target']
        
        # Modelagem (base learners)
        model = DecisionTreeClassifier()
        model.fit(X_train, y_train)
        
        # Adicionando os resultados do modelo ao dicionário para agregação das predições
        y_pred_bagging.update({i:model.predict(X_test[X_train.columns])})
    
    # Aggregating
    y_pred = (pd.DataFrame(y_pred_bagging)
                .mode(axis=1)  # Agregando o valor com maior número de aparições nas predições dos modelos
                .rename(columns={0:'y_pred'}))
 
    # Resultados
    print(model)
    print('Accuracy score:', accuracy_score(y_true=y_test, 
                                            y_pred=y_pred['y_pred']
                                           ))

    return pd.concat(objs=[y_test.reset_index(drop=True), 
                           y_pred['y_pred'].astype(int)], 
                     axis=1)

rf_classifier(num_bootstrap_samples=10, df=df, test_size=0.33)

DecisionTreeClassifier()
Accuracy score: 0.94


,y_test,y_pred
0,2,2
1,0,0
2,0,0
3,1,1
4,0,0
5,0,0
6,2,2
7,0,0
8,2,2
9,0,0


In [3]:
# Exemplo da técnica Random Forest para problemas de regressão:

X = load_diabetes().data
y = load_diabetes().target

df = pd.DataFrame(X, columns=load_diabetes().feature_names)
df['target'] = y

def rf_regressor(df:pd.DataFrame, 
                 num_bootstrap_samples:int=3,  # Parâmetro da função que define a quantidade de amostragens para treinamento
                 test_size:float=0.25
                ) -> pd.DataFrame:
    
    df_train, df_test = train_test_split(df, test_size=test_size)
    
    X_test = df_test.drop(['target'], axis=1)
    y_test = df_test['target'].rename('y_test')
    
    # Dicionário para os resultados das predições de cada modelo
    y_pred_bagging = {}

    for i in range(num_bootstrap_samples):
        # Bootstrap
        df_train = df_train.sample(n=len(df_train), 
                                   replace=True)  # Amostragem COM reposição

        X_train = df_train.drop(['target'], axis=1)
        # Feature selection
        X_train = X_train.sample(n=round(X_train.shape[1]/3),  # Cálculo da quantidade de variáveis dividida por 3
                                 axis=1)
        
        y_train = df_train['target']

        # Modelagem (base learners)
        model = DecisionTreeRegressor()
        model.fit(X_train, y_train)
        
        # Adicionando os resultados do modelo ao dicionário para agregação das predições
        y_pred_bagging.update({i:model.predict(X_test[X_train.columns])})

    # Aggregating
    y_pred = (pd.DataFrame(y_pred_bagging)
                .mean(axis=1)  # Agregando as predições dos modelos baseando n a média dos resultados
                .rename('y_pred'))
 
    # Resultados
    print(model)
    print('Mean squared error:', mean_squared_error(y_true=y_test, 
                                                   y_pred=y_pred))
    print('Coefficient of determination:', r2_score(y_true=y_test, 
                                                    y_pred=y_pred))
    
    return pd.concat(objs=[y_test.reset_index(drop=True), 
                           y_pred], 
                     axis=1)
    
rf_regressor(num_bootstrap_samples=10, df=df, test_size=0.33)

DecisionTreeRegressor()
Mean squared error: 4499.433081769329
Coefficient of determination: 0.30987046585364


,y_test,y_pred
0,220.0,233.500000
1,163.0,175.200000
2,102.0,127.700000
3,170.0,85.300000
4,59.0,82.200000
...,...,...
141,55.0,135.100000
142,70.0,112.766667
143,191.0,141.200000
144,84.0,159.000000


---